# README.md

#one pt #83639 was assgined missing code and should be CRC

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [Assign drug use status per row](#second-bullet)

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [50]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

import matplotlib.pyplot as plt
from matplotlib import pyplot
from pylab import *

%matplotlib inline  
import seaborn as sns

In [56]:
#cancer_type='CRC'

df2_pan=pd.read_excel("./data/2021_0309_CancerTypebyOnctree_vsCancerTypeByDx_reviewed_Yuqin_Mar22.xlsx",header=0)
df2=df2_pan[(df2_pan["person_id"] == 83639)]

df_crc=pd.read_csv("./data/genomics_CRC_temp_Mar26.csv",header=0)
df_crc_columns=df_crc.columns.values.tolist()

#cancer_type='Bowel'
df1=pd.read_csv("./data/pt_83639_pan_full_medication_list_updated_202103300954.csv",header=0)

#df2=pd.read_excel("./data/pt_list_all_rows.xlsx", header=0) 
#####pts in the database
df3=pd.read_csv("./data/distint_person_id_pan_full_medication_list_updated_202103231646.csv", header=0) 

In [57]:
df2[:3]

,person_id,gene,variant,variant_type,is_clinically_significant,pored_tumor_type_displayname,pored_cancer_type_name,oncsuite_sample_site,oncotree_code,tested_date,cplus_cancer_type_name,dxdate,dx_year,dx_month,yrdiff_dx_test,Comparision,FinalCancerTypeAssigned,Unnamed: 17
3042,83639,TP53,c.376-1G>T,SNV,S,Colon,CRC,Bowel,APAD,2020-07-10,BCA,2018-09-22,2018,9.0,2,Diff,code missing,NaN
3043,83639,RB1,p.I680V,SNV,US,Colon,CRC,Bowel,APAD,2020-07-10,BCA,2018-09-22,2018,9.0,2,Diff,code missing,NaN
3044,83639,ARID1A,p.H170Y,SNV,US,Colon,CRC,Bowel,APAD,2020-07-10,BCA,2018-09-22,2018,9.0,2,Diff,code missing,NaN


In [6]:
#drug_cat=df1["category"].to_list()
#unique_drug_cat=set(drug_cat)
#unique_drug_cat

In [7]:
print(shape(df1))

(421010, 31)


In [8]:
df2[:3]

,person_id,gene,variant,variant_type,is_clinically_significant,pored_tumor_type_displayname,pored_cancer_type_name,oncsuite_sample_site,oncotree_code,tested_date,cplus_cancer_type_name,dxdate,dx_year,dx_month,yrdiff_dx_test,Comparision,FinalCancerTypeAssigned,Unnamed: 17
3042,83639,TP53,c.376-1G>T,SNV,S,Colon,CRC,Bowel,APAD,2020-07-10,BCA,2018-09-22,2018,9.0,2,Diff,code missing,NaN
3043,83639,RB1,p.I680V,SNV,US,Colon,CRC,Bowel,APAD,2020-07-10,BCA,2018-09-22,2018,9.0,2,Diff,code missing,NaN
3044,83639,ARID1A,p.H170Y,SNV,US,Colon,CRC,Bowel,APAD,2020-07-10,BCA,2018-09-22,2018,9.0,2,Diff,code missing,NaN


In [59]:
shape(df2)

(3, 18)

In [60]:
#get the drug list
drug_list=df1["cancer_drug"].to_list()
result_row_list=df2["person_id"].to_list()
pts_in_database=df3["person_id"].to_list()

In [63]:
print("row counts")
print(len(result_row_list))
unique_result_pt_list=set(result_row_list)
print("unique pt counts")
print(len(unique_result_pt_list))

print("unique pt in database")
unique_result_pt_list_in_database=set(pts_in_database)
print(len(unique_result_pt_list_in_database))

#print("overlapping pt in database")
#are all the pts in the database ? 
overlap_list=[x for x in unique_result_pt_list if x in  unique_result_pt_list_in_database]
#print(len(overlap_list))

#not avail pt list
pt_nan_list=[x for x in unique_result_pt_list if x not in  unique_result_pt_list_in_database]
print("pts not in database")
print(len(pt_nan_list))

row counts
3
unique pt counts
1
unique pt in database
191383
pts not in database
0


In [64]:
pt_nan_list

[]

In [65]:
#for this list of rows get the dx etc info 
result_row_list[:5]

[83639, 83639, 83639]

In [92]:
#unique drug list 
df1_drug=df1.groupby(["cancer_drug"]).size().reset_index(name='counts')
unique_drug_list=df1_drug["cancer_drug"].to_list()
print(len(unique_drug_list))
unique_drug_list

9


['Anastrozole',
 'Capecitabine',
 'Carboplatin',
 'Docetaxel',
 'Mercaptopurine',
 'Paclitaxel',
 'Pertuzumab',
 'Tamoxifen',
 'Trastuzumab']

In [67]:
#df1_drug=df1.groupby(["cancer_drug"]).size().reset_index(name='counts')
#83639

df1_pt=df1[(df1["person_id"] == 83639)] 
pt_drug=df1_pt.cancer_drug.to_list()

In [68]:
pt_drug
df1_pt

,person_id,cancer_drug,matched_text,category,result_source,note_id,note_date,sent_id,evidence,date_year,date_month,date_day
0,83639,Mercaptopurine,6mp,Chemotherapy,Notes,20184322.0,2020-10-06,653409583.0,History of Present Illness Ms. Margaret Lanz...,NaN,NaN,NaN
1,83639,Paclitaxel,paclitaxel,Chemotherapy,Notes,14006087.0,2019-02-07,345182192.0,"========== TREATMENT: -SURGERY: 10/16/2018, ...",NaN,NaN,NaN
2,83639,Trastuzumab,trastuzumab,Targeted,Notes,16212245.0,2020-03-04,527891265.0,TCH: The TCH regimen consists of docetaxel (ta...,NaN,NaN,NaN
3,83639,Trastuzumab,trastuzumab,Targeted,Notes,11874998.0,2019-02-28,344598690.0,TCH: The TCH regimen consists of docetaxel (ta...,NaN,NaN,NaN
4,83639,Trastuzumab,herceptin,Targeted,Notes,18996972.0,2019-03-21,580805309.0,12/13/2018: vision changes have fully resolved...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1059,83639,Docetaxel,taxotere,Chemotherapy,Notes,11870674.0,2019-02-21,344482857.0,TCH: The TCH regimen consists of docetaxel (ta...,NaN,NaN,NaN
1060,83639,Trastuzumab,trastuzumab,Targeted,Notes,14057072.0,2019-07-25,344546962.0,"Side effects include, but are not limited to, ...",NaN,NaN,NaN
1061,83639,Trastuzumab,trastuzumab,Targeted,Notes,19732356.0,2020-07-30,633358748.0,History of Present Illness Ms. Margaret Lanz...,NaN,NaN,NaN
1062,83639,Mercaptopurine,6mp,Chemotherapy,Notes,19487579.0,2020-06-15,617089546.0,Was started on Prednisone 40 mg as well as 6MP...,NaN,NaN,NaN


In [44]:
#unique_drug_list

In [85]:
column_names=df_crc_columns[2:]
print(len(column_names))

crc_used_drug=[x[:-4] for x in column_names]
#crc_used_drug

88


['Acitretin',
 'Aflibercept',
 'Alpelisib',
 'Anastrozole',
 'Arsenic Trioxide',
 'Atezolizumab',
 'Avelumab',
 'Azathioprine',
 'Bevacizumab',
 'Bexarotene',
 'Bicalutamide',
 'Binimetinib',
 'Cabozantinib',
 'Capecitabine',
 'Carboplatin',
 'Cetuximab',
 'Cisplatin',
 'Crizotinib',
 'Cyclophosphamide',
 'Cyclosporine',
 'Cytarabine',
 'Dabrafenib',
 'Dasatinib',
 'Degarelix',
 'Docetaxel',
 'Doxorubicin',
 'Dupilumab',
 'Encorafenib',
 'Entrectinib',
 'Enzalutamide',
 'Eribulin',
 'Ethinylestradiol',
 'Etoposide',
 'Filgrastim',
 'Floxuridine',
 'Fluorouracil',
 'Gemcitabine',
 'Goserelin',
 'Hydroxyurea',
 'Idarubicin',
 'Idelalisib',
 'Imiquimod',
 'Interferon Alfa-2b',
 'Interleukin-2',
 'Iobenguane',
 'Ipilimumab',
 'Irinotecan',
 'Isotretinoin',
 'Ketoconazole',
 'Lanreotide',
 'Lapatinib',
 'Larotrectinib',
 'Letrozole',
 'Leuprolide',
 'Medroxyprogesterone Acetate',
 'Megestrol Acetate',
 'Mercaptopurine',
 'Methotrexate',
 'Mitomycin',
 'Nivolumab',
 'Octreotide',
 'Olaparib'

In [17]:
#result_row_list[:5]

# Assign drug  status <a class="anchor" id="second-bullet"></a>

In [86]:
#make a function 
### approach pick a drug, do the Y/N, get the column name and info; 
### for all the row in result_row_list; 
### then combine all columns

def assign_status(drug):
    result_status_list=[]

    for y in result_row_list: 
        
        if y in pt_nan_list:
            status='NaN'
        
        if y in overlap_list:
            
            df1_pt=df1[(df1["person_id"] == y)] 
            pt_drug=df1_pt.cancer_drug.to_list()
    
            if len(pt_drug)==0:
                status='N'
                print("drug=0")
            if len(pt_drug)>0:
                
                if drug in pt_drug:
                    status='Y'
                if drug not in pt_drug:
                    status='N'            
                    #print("drug not in pt_drug")
        result_pt=[y,status]
        result_status_list.append(result_pt)  
    
    return result_status_list

In [87]:
df=pd.DataFrame(result_row_list).reset_index()
df.columns=(['index','person_id'])

In [88]:
for drug in crc_used_drug:
    column_n=drug+"_Y/N" 
    df_results=pd.DataFrame(assign_status(drug)) 
    df_results.columns=(['person_id', column_n])
    #if df_results[column_n].str.contains('Y'):       
    df[column_n]=df_results[column_n]

In [89]:
df[:5]

,index,person_id,Acitretin_Y/N,Aflibercept_Y/N,Alpelisib_Y/N,Anastrozole_Y/N,Arsenic Trioxide_Y/N,Atezolizumab_Y/N,Avelumab_Y/N,Azathioprine_Y/N,...,Trametinib_Y/N,Trastuzumab_Y/N,Tretinoin_Y/N,Trifluridine_Y/N,Trifluridine + Tipiracil Hydrochloride_Y/N,Vandetanib_Y/N,Vemurafenib_Y/N,Vinorelbine_Y/N,Vorinostat_Y/N,Yttrium Y-90_Y/N
0,0,83639,N,N,N,Y,N,N,N,N,...,N,Y,N,N,N,N,N,N,N,N
1,1,83639,N,N,N,Y,N,N,N,N,...,N,Y,N,N,N,N,N,N,N,N
2,2,83639,N,N,N,Y,N,N,N,N,...,N,Y,N,N,N,N,N,N,N,N


In [90]:
shape(df)

(3, 90)

In [91]:
df.to_csv("./data/genomics_MISSINGCODE_temp.csv", encoding='utf-8', index=False)